In [1]:
from pyspark import SparkConf, SparkContext

conf = (SparkConf()
        .setAppName("LR_3_Analyze_russian_dict")
        .setMaster("local[*]")
)
sc = SparkContext(conf=conf)
rdd = sc.textFile("Rus_dict.txt")

### Статистика по словам:

- Посчитайте общее количество слов в словаре.
- Найдите Топ-5 самых длинных слов.
- Найдите Топ-5 самых коротких слов.

In [4]:
print(f"общее количество слов в словаре:", rdd.count())

# поиск 5-ти самых длинных слов
len_words = rdd.map(lambda x: (x, len(x)))
print(f"TOP-5 самых длинных слов")
for w, l in len_words.top(5, lambda x: x[1]):
    print(f"{w}: {l}")

# поиск 5-ти самых коротких слов
print(f"TOP-5 самых коротких слов")
for w, l in len_words.top(5, lambda x: -x[1]):
    print(f"{w}: {l}")

общее количество слов в словаре: 51301
TOP-5 самых длинных слов
человеконенавистничество: 24
интернационализирование: 23
неусовершенствованность: 23
переосвидетельствование: 23
адмиралтейств-коллегия: 22
TOP-5 самых коротких слов
ад: 2
аз: 2
аи: 2
ар: 2
ас: 2


### Анализ по длине слов:  
- Посчитайте, сколько слов имеют каждую конкретную длину (например, слов длиной 3: X, слов длиной 5: Y).  
   Отсортируйте результат по длине.
- Определите длину слова, которая встречается чаще всего.

In [8]:
# Подсчет кол-ва слов, имеющих конкретную длину
len_lst = rdd.map(lambda x: (len(x), 1))
len_lst = len_lst.reduceByKey(lambda a, b: a + b)
for l, cnt in len_lst.sortByKey().collect():
    print(f"{l}: {cnt}")

print(f"Длина слова, которая встречается чаще всего: {len_lst.max(lambda x: x[1])[0]}")

2: 54
3: 472
4: 1607
5: 3483
6: 4836
7: 6335
8: 6885
9: 6612
10: 5501
11: 4591
12: 3527
13: 2774
14: 1994
15: 1123
16: 692
17: 364
18: 227
19: 117
20: 60
21: 29
22: 14
23: 3
24: 1
Длина слова, которая встречается чаще всего: 8


### Символьный анализ:
- Посчитайте, сколько слов содержат букву 'ё'
- Отфильтруйте и выведите все слова, которые являются палиндромами
   (читаются одинаково вперед и назад, например, "шалаш", "заказ").

In [10]:
rdd_filtered_e = rdd.filter(lambda x: "ё" in x)
cnt_e = rdd_filtered_e.count()
print(f"Количество слов содержащих букву ё: {cnt_e}")

rdd_filtered_poli = rdd.filter(lambda x: x == x[::-1])
print("Полиндромы:")
for word in rdd_filtered_poli.collect():
    print(word)


Количество слов содержащих букву ё: 2111
Полиндромы:
ага
боб
дед
довод
доход
заказ
кабак
казак
киник
кок
колок
комок
косок
коток
лал
мадам
мим
наган
нойон
око
поп
потоп
пуп
радар
репер
ротатор
ротор
тат
тет-а-тет
топот
тот
тут
ушу
шалаш
шиш


In [11]:
sc.stop()

## Решение DataFrame

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, sum, min, max, length, avg, round, count, substring
from pyspark.sql.types import StructType, StructField, StringType

In [13]:
spark = SparkSession.builder \
        .appName("LR_3_Analyze_russian_dict") \
        .master("local[*]") \
        .getOrCreate()
file_schema = StructType([
    StructField("word", StringType(), True)
])

file_path = "Rus_dict.txt"
df_dict = spark.read.csv(
    file_path,
    header=False,
    schema=file_schema
)

### Статистика по словам:

- Посчитайте общее количество слов в словаре.
- Найдите Топ-5 самых длинных слов.
- Найдите Топ-5 самых коротких слов.

In [23]:

print(f"Общее количество слов в словаре: {df_dict.count()}")

# Найдите Топ-5 самых длинных слов.
df_len_words = df_dict.groupBy(col("word")).agg(
    length(col("word")).alias("length_word")
).orderBy(col("length_word").desc())

print("Топ 5 самых длинных слов:")
df_len_words.show(5, truncate=False)

# Найдите Топ-5 самых коротких слов.
print("Топ 5 самых коротких слов:")
df_len_words.orderBy(col("length_word")).show(5, truncate=False)

Общее количество слов в словаре: 51301
Топ 5 самых длинных слов:
+------------------------+-----------+
|word                    |length_word|
+------------------------+-----------+
|человеконенавистничество|24         |
|переосвидетельствование |23         |
|интернационализирование |23         |
|неусовершенствованность |23         |
|неудовлетворительность  |22         |
+------------------------+-----------+
only showing top 5 rows

Топ 5 самых коротких слов:
+----+-----------+
|word|length_word|
+----+-----------+
|си  |2          |
|ох  |2          |
|ар  |2          |
|ям  |2          |
|ир  |2          |
+----+-----------+
only showing top 5 rows



### Анализ по длине слов:  
- Посчитайте, сколько слов имеют каждую конкретную длину (например, слов длиной 3: X, слов длиной 5: Y).  
   Отсортируйте результат по длине.
- Определите длину слова, которая встречается чаще всего.

In [34]:
df_dict_len = df_len_words.groupBy(col("length_word")).agg(
    count(col("length_word")).alias("cnt_len")
).orderBy(col("cnt_len").desc())
df_dict_len.show()

# Длина слова, которая встречается чаще всего
len_request = df_dict_len.limit(1).collect()[0][0]
print(f"Длина слова, которая встречается чаще всего: {len_request}")

+-----------+-------+
|length_word|cnt_len|
+-----------+-------+
|          8|   6885|
|          9|   6612|
|          7|   6335|
|         10|   5501|
|          6|   4836|
|         11|   4591|
|         12|   3527|
|          5|   3483|
|         13|   2774|
|         14|   1994|
|          4|   1607|
|         15|   1123|
|         16|    692|
|          3|    472|
|         17|    364|
|         18|    227|
|         19|    117|
|         20|     60|
|          2|     54|
|         21|     29|
+-----------+-------+
only showing top 20 rows

Длина слова, которая встречается чаще всего: 8


### Символьный анализ:
- Посчитайте, сколько слов содержат букву 'ё'
- Отфильтруйте и выведите все слова, которые являются палиндромами
   (читаются одинаково вперед и назад, например, "шалаш", "заказ").

In [46]:
from pyspark.sql.functions import reverse
# Подсчет слов, содержащих букву 'ё'
df_with_e = df_dict.filter(col("word").contains("ё"))
cnt_words_e = df_with_e.count()
print(f"Кол-во слов, содержаших букву ё: {cnt_words_e}")

# поиск полиндромов
df_polindroms = df_dict.filter(col("word") == reverse(col("word")))
print("Полиндромы:")
df_polindroms.show()

Кол-во слов, содержаших букву ё: 2111
Полиндромы:
+-----+
| word|
+-----+
|  ага|
|  боб|
|  дед|
|довод|
|доход|
|заказ|
|кабак|
|казак|
|киник|
|  кок|
|колок|
|комок|
|косок|
|коток|
|  лал|
|мадам|
|  мим|
|наган|
|нойон|
|  око|
+-----+
only showing top 20 rows



In [47]:
spark.stop()